In [10]:
using System.IO;

In [11]:

//# Start of the data on the page
readonly var FIL_PAGE_DATA = 38;


readonly var FIL_PAGE_OFFSET = 4;// # page offset inside space
readonly var FIL_PAGE_TYPE = 24 ;//# File page type

//# Types of an undo log segment */
readonly var TRX_UNDO_INSERT = 1;
readonly var TRX_UNDO_UPDATE = 2;

//# On a page of any file segment, data may be put starting from this offset
readonly var FSEG_PAGE_DATA = FIL_PAGE_DATA;

//# The offset of the undo log page header on pages of the undo log
readonly var TRX_UNDO_PAGE_HDR = FSEG_PAGE_DATA;

readonly var PAGE_LEVEL = 26;	//#level of the node in an index tree; the leaf level is the level 0 */
				
readonly var innodb_page_type=new Dictionary<string, string>(){
	{"0000","Freshly Allocated Page"},
	{"0002","Undo Log Page"},
	{"0003","File Segment inode"},
	{"0004","Insert Buffer Free List"},
	{"0005","Insert Buffer Bitmap"},
	{"0006","System Page"},
	{"0007","Transaction system Page"},
	{"0008","File Space Header"},
	{"0009","extend description page"},
	{"000a","Uncompressed BLOB Page"},
	{"000b","1st compressed BLOB Page"},
	{"000c","Subsequent compressed BLOB Page"},
	{"45bf","B-tree Node"},
    {"45bd","Tablespace SDI Index page"}
	};
	
readonly var innodb_page_direction=new Dictionary<string, string>(){
	{"0000","Unknown(0x0000)"},
	{"0001", "Page Left"},
	{"0002", "Page Right"},
	{"0003", "Page Same Rec"},
	{"0004", "Page Same Page"},
	{"0005", "Page No Direction"},
	{"ffff", "Unkown2(0xffff)"}
};


readonly var INNODB_PAGE_SIZE=1024*16; //# InnoDB Page 16K

In [12]:
public string MachReadFromN(byte[] page,int start_offset,int length)
{
	var ret = page[start_offset..(start_offset+length)];
	return Convert.ToHexString(ret);
}
public void GetInnodbPageType()
{
	var f=new FileInfo(@"E:\MySQL\Data\admin\tb_users.ibd");



	var fsize = f.Length/INNODB_PAGE_SIZE;

	dynamic ret = new{};
	var read = new BinaryReader(f.OpenRead());
	for (int i = 0;i<fsize;i++)
	{
		
		var page = read.ReadBytes(INNODB_PAGE_SIZE);
		var page_offset = MachReadFromN(page,FIL_PAGE_OFFSET,4);
		var page_pre = MachReadFromN(page,FIL_PAGE_OFFSET+4,4);
		var page_next = MachReadFromN(page,FIL_PAGE_OFFSET+8,4);
		var page_lsn = MachReadFromN(page,FIL_PAGE_OFFSET+12,8);
		
		var page_type = MachReadFromN(page,FIL_PAGE_TYPE,2);
		var flush_lsn = MachReadFromN(page,FIL_PAGE_TYPE+2,8);
		var space_id = MachReadFromN(page,FIL_PAGE_TYPE+10,4);
		var data = MachReadFromN(page,38,INNODB_PAGE_SIZE-8);

		if (page_type == "45bf")
		{display("-----");
		    var	page_level = MachReadFromN(page,FIL_PAGE_DATA+PAGE_LEVEL,2);
			display(string.Format("page offset [{}], page type <{}>, pre page [{}], next page [{}], page level <{}> , flush_lsn <{}> , space_id <{}>, data {} \n",page_offset,innodb_page_type[page_type],page_pre,page_next,page_level,flush_lsn,space_id,data));
		}
		else
		{display("-----");
			display(string.Format("page offset [{}], page type <{}>, pre page [{}], next page [{}], flush_lsn <{}> , space_id <{}>, data {} \n"  ,page_offset,innodb_page_type[page_type],page_pre,page_next,flush_lsn,space_id,data));
		}
			
		
	}
}
GetInnodbPageType();

0

Error: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'length')
   at System.Range.ThrowArgumentOutOfRangeException()
   at System.Runtime.CompilerServices.RuntimeHelpers.GetSubArray[T](T[] array, Range range)
   at Submission#13.MachReadFromN(Byte[] page, Int32 start_offset, Int32 length)
   at Submission#13.GetInnodbPageType()
   at Submission#13.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)